# Notebook for collection of data + EDA
# James Yu

In [12]:
import pandas as pd
from random import sample
import sys
import json

## Looking at arXiv dataset from Kaggle

In [13]:
fp1 = '../data/arxivData.json'

In [14]:
# arXiv dataset from Kaggle https://www.kaggle.com/neelshah18/arxivdataset
df1 = pd.read_json(fp1)
df1.head()

,author,day,id,link,month,summary,tag,title,year
0,"[{'name': 'Ahmed Osman'}, {'name': 'Wojciech S...",1,1802.00209v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,We propose an architecture for VQA which utili...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",Dual Recurrent Attention Units for Visual Ques...,2018
1,"[{'name': 'Ji Young Lee'}, {'name': 'Franck De...",12,1603.03827v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",3,Recent approaches based on artificial neural n...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Sequential Short-Text Classification with Recu...,2016
2,"[{'name': 'Iulian Vlad Serban'}, {'name': 'Tim...",2,1606.00776v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",6,We introduce the multiresolution recurrent neu...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Multiresolution Recurrent Neural Networks: An ...,2016
3,"[{'name': 'Sebastian Ruder'}, {'name': 'Joachi...",23,1705.08142v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",5,Multi-task learning is motivated by the observ...,"[{'term': 'stat.ML', 'scheme': 'http://arxiv.o...",Learning what to share between loosely related...,2017
4,"[{'name': 'Iulian V. Serban'}, {'name': 'Chinn...",7,1709.02349v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",9,We present MILABOT: a deep reinforcement learn...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",A Deep Reinforcement Learning Chatbot,2017


In [15]:
#df1.value_counts('year')

In [16]:
#df1.value_counts('month')

In [17]:
agg_titles_by_year = df1.groupby('year')['title'].apply(' '.join)
agg_titles_by_year.head()

year
1993    Dynamic Backtracking A Market-Oriented Program...
1994    Teleo-Reactive Programs for Agent Control Lear...
1995    Solving Multiclass Learning Problems via Error...
1996    The Design and Experimental Analysis of Algori...
1997    Machine Learning of User Profiles: Representat...
Name: title, dtype: object

In [18]:
agg_summ_by_year = df1.groupby('year')['summary'].apply(' '.join)
agg_summ_by_year.head()

year
1993    Because of their occasional need to return to ...
1994    A formalism is presented for computing and org...
1995    Multiclass learning problems involve finding a...
1996    Many applications -- from planning and schedul...
1997    As more information becomes available electron...
Name: summary, dtype: object

In [19]:
agg_all_by_year = df1.groupby('year')['title'].apply(' '.join) + ' ' + df1.groupby('year')['summary'].apply(' '.join)
agg_all_by_year.head()

year
1993    Dynamic Backtracking A Market-Oriented Program...
1994    Teleo-Reactive Programs for Agent Control Lear...
1995    Solving Multiclass Learning Problems via Error...
1996    The Design and Experimental Analysis of Algori...
1997    Machine Learning of User Profiles: Representat...
dtype: object

In [20]:
agg_all_by_year[1993][:500]

'Dynamic Backtracking A Market-Oriented Programming Environment and its Application to\n  Distributed Multicommodity Flow Problems An Empirical Analysis of Search in GSAT The Difficulties of Learning Logic Programs with Cut Software Agents: Completing Patterns and Constructing User Interfaces Decidable Reasoning in Terminological Knowledge Representation Systems Because of their occasional need to return to shallow points in a search\ntree, existing backtracking methods can sometimes erase meaningf'

In [21]:
agg_all_by_year[1993][:500].replace('\n', ' ')

'Dynamic Backtracking A Market-Oriented Programming Environment and its Application to   Distributed Multicommodity Flow Problems An Empirical Analysis of Search in GSAT The Difficulties of Learning Logic Programs with Cut Software Agents: Completing Patterns and Constructing User Interfaces Decidable Reasoning in Terminological Knowledge Representation Systems Because of their occasional need to return to shallow points in a search tree, existing backtracking methods can sometimes erase meaningf'

## Looking at DBLP v13 dataset

In [22]:
fp2 = '../data/dblpv13.json'

In [23]:
# DBLP v13 dataset
# file size is 16.1 GB
# Can try to read in as smaller pieces, or write out the data into .txt files by year

#df2 = pd.read_json(fp2)
#df2.head()

In [168]:
def process_json(inpath):
    """
    Process the inpath of DBLP v13 dataset
    Outputs a list of dictionaries (each dictionary is a paper)

    >>> process_json('../data/dblpv13.json')
    """
    count = 0
    data = []
    first_line = True

    with open(inpath) as f:
        content = ''
        # Keeps track of the start of processing a block (individual paper)
        start_ind = False
        for line in f:
            # Indicates start of block
            if line == '{ \n':
                start_ind = True
            # Indicates end of block
            if line == '},\n':
                # Doesn't process the first block (only contains info about structure)
                if first_line:
                    first_line = False
                    content = ''
                    continue
                # Cleans up formatting of block
                content += '}\n'
                content = content.replace(' :', ':').replace(',', ', ')
                data.append(json.loads(content))
                start_ind = False
                content = ''

            if start_ind:
                # Cleans up line to prevent issues with json.loads()
                line = line.strip()
                if 'NumberInt' in line:
                    line = line.replace('NumberInt(', '')
                    line = line.strip()[:-2] + ','
                if line[-3:] == ': ,':
                    line = line.replace(': ,', ': null')
                content += line

            count += 1
            if count > 200:
                break
    return data

In [169]:
process_json(fp2)

[{'_id': '53e99784b7602d9701f3e133',
  'title': 'The relationship between canopy parameters and spectrum of winter wheat under different irrigations in Hebei Province.',
  'authors': [{'_id': '53f45728dabfaec09f209538', 'name': 'Peijuan Wang'},
   {'_id': '5601754345cedb3395e59457', 'name': 'Jiahua Zhang'},
   {'_id': '53f38438dabfae4b34a08928', 'name': 'Donghui Xie'},
   {'_id': '5601754345cedb3395e5945a', 'name': 'Yanyan Xu'},
   {'_id': '53f43d25dabfaeecd6995149', 'name': 'Yun Xu'}],
  'venue': {'_id': '53a7297d20f7420be8bd4ae7',
   'name_d': 'International Geoscience and Remote Sensing Symposium',
   'type': 0,
   'raw': 'IGARSS'},
  'year': 2011,
  'keywords': ['canopy parameters',
   'canopy spectrum',
   'different soil water content control',
   'winter wheat',
   'irrigation',
   'hydrology',
   'radiometry',
   'moisture',
   'indexes',
   'vegetation',
   'indexation',
   'dry weight',
   'soil moisture',
   'water content',
   'indexing terms',
   'spectrum',
   'natural di

# Looking at DBLP v10 dataset

In [26]:
fp3 = ['../data/dblp-v10/' + 'dblp-ref-' + str(x) + '.json' for x in range(4)]
fp3

['../data/dblp-v10/dblp-ref-0.json',
 '../data/dblp-v10/dblp-ref-1.json',
 '../data/dblp-v10/dblp-ref-2.json',
 '../data/dblp-v10/dblp-ref-3.json']

In [68]:
file = open(fp3[0])

fp = '2015.txt'
#open(fp, 'w').close()

for l in file:
    x = l
    data = json.loads(l)
    print(data)
    # Only info we need: abstract, title, year
    year = data['year']
    outpath = str(year) + '.txt'
    #outfile = open(outpath, 'a', encoding='utf-8')
    break
file.close()

{'abstract': 'The purpose of this study is to develop a learning tool for high school students studying the scientific aspects of information and communication net- works. More specifically, we focus on the basic principles of network proto- cols as the aim to develop our learning tool. Our tool gives students hands-on experience to help understand the basic principles of network protocols.', 'authors': ['Makoto Satoh', 'Ryo Muramatsu', 'Mizue Kayama', 'Kazunori Itoh', 'Masami Hashimoto', 'Makoto Otani', 'Michio Shimizu', 'Masahiko Sugimoto'], 'n_citation': 0, 'references': ['51c7e02e-f5ed-431a-8cf5-f761f266d4be', '69b625b9-ebc5-4b60-b385-8a07945f5de9'], 'title': 'Preliminary Design of a Network Protocol Learning Tool Based on the Comprehension of High School Students: Design by an Empirical Study Using a Simple Mind Map', 'venue': 'international conference on human-computer interaction', 'year': 2013, 'id': '00127ee2-cb05-48ce-bc49-9de556b93346'}


In [69]:
x

'{"abstract": "The purpose of this study is to develop a learning tool for high school students studying the scientific aspects of information and communication net- works. More specifically, we focus on the basic principles of network proto- cols as the aim to develop our learning tool. Our tool gives students hands-on experience to help understand the basic principles of network protocols.", "authors": ["Makoto Satoh", "Ryo Muramatsu", "Mizue Kayama", "Kazunori Itoh", "Masami Hashimoto", "Makoto Otani", "Michio Shimizu", "Masahiko Sugimoto"], "n_citation": 0, "references": ["51c7e02e-f5ed-431a-8cf5-f761f266d4be", "69b625b9-ebc5-4b60-b385-8a07945f5de9"], "title": "Preliminary Design of a Network Protocol Learning Tool Based on the Comprehension of High School Students: Design by an Empirical Study Using a Simple Mind Map", "venue": "international conference on human-computer interaction", "year": 2013, "id": "00127ee2-cb05-48ce-bc49-9de556b93346"}\n'

In [67]:
data

{'abstract': 'The purpose of this study is to develop a learning tool for high school students studying the scientific aspects of information and communication net- works. More specifically, we focus on the basic principles of network proto- cols as the aim to develop our learning tool. Our tool gives students hands-on experience to help understand the basic principles of network protocols.',
 'authors': ['Makoto Satoh',
  'Ryo Muramatsu',
  'Mizue Kayama',
  'Kazunori Itoh',
  'Masami Hashimoto',
  'Makoto Otani',
  'Michio Shimizu',
  'Masahiko Sugimoto'],
 'n_citation': 0,
 'references': ['51c7e02e-f5ed-431a-8cf5-f761f266d4be',
  '69b625b9-ebc5-4b60-b385-8a07945f5de9'],
 'title': 'Preliminary Design of a Network Protocol Learning Tool Based on the Comprehension of High School Students: Design by an Empirical Study Using a Simple Mind Map',
 'venue': 'international conference on human-computer interaction',
 'year': 2013,
 'id': '00127ee2-cb05-48ce-bc49-9de556b93346'}